<div style="background:#634C44; color:white; padding:12px; box-sizing:border-box; border-radius:4px;">

</div>

# Text Preprocessing

> **Author:** Jason Cruz  
  **Last updated:** 11/16/2025  
  **Python version:** 3.12  
  **Project:** Fiscal Tone

---

## 📌 Summary
Welcome to the **Fiscal Tone** Text Preprocessing notebook! This notebook will guide you through the **step-by-step process** of 


### What will this notebook help you achieve?
1. **Downloading PDFs** from the BCRP Weekly Reports (WR).
2. **Generating PDF inputs** by shortening them to focus on key pages containing GDP growth rate tables.
3. **Cleaning-up extracted data** to ensure it's usable and building RTD.
4. **Concatenating RTD** from different years and frequencies (monthly, quarterly, annual).
5. **Updating metadata** for storing base years changes and other revisions-based information.
6. **Converting RTD** to releases dataset for econometric analysis.

🌐 **Main Data Source:** [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html) (📰 WR, from here on)  
For any questions or issues, feel free to reach out via email: [Jason 📨](mailto:jj.cruza@up.edu.pe)

---

### ⚙️ Initial Set-up

Before preprocessing the new GDP releases data, we need to perform some initial set-up steps:

1. 🧰 **Import helper functions** from `gdp_rtd_pipeline.py` that are required for this notebook.
2. 🛢️ **Connect to the PostgreSQL database** that will contain GDP revisions datasets. _(This step is pending: direct access will be provided via ODBC or other methods, allowing users to connect from any software or programming language.)_
3. 📂 **Create necessary folders** to store inputs, outputs, logs, and screenshots.

In [ ]:
#from cf_mef_functions.py import * 

> 🚧 Although the second step (database connection) is pending, the notebook currently works using **flat files (CSV)**. These CSV files will **not be saved in GitHub** as they are included in the `.gitignore` to ensure no data is stored publicly. Users can be confident that no data will be stored on GitHub. The notebook **automatically generates the CSV files**, giving users direct access to the dataset on their own systems. The data is created on the fly and can be saved locally for further use.

### 🧰 Import helper functions

This notebook relies on a set of helper functions found in the script `gdp_rtd_pipeline.py`. These functions will be used throughout the notebook, so please ensure you have them ready by running the line of code below.

In [ ]:
from text_preprocessing_pipeline import *

> 🛠️ **Libraries:** Before you begin, please ensure that you have the required libraries installed and imported. See all the libraries you need section by section in `gdp_rtd_pipeline.py`.

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

**Check out Python information**

In [1]:
import sys
import platform

print("🐍 Python Information")
print(f"  Version  : {sys.version.split()[0]}")
print(f"  Compiler : {platform.python_compiler()}")
print(f"  Build    : {platform.python_build()}")
print(f"  OS       : {platform.system()} {platform.release()}")

🐍 Python Information
  Version  : 3.12.11
  Compiler : MSC v.1929 64 bit (AMD64)
  Build    : ('main', 'Jun  5 2025 12:58:53')
  OS       : Windows 10


### 📂 Create necessary folders

We will start by creating the necessary folders to store the data at various stages of processing. The following code ensures all required directories exist, and if not, it creates them.

In [2]:
import os

In [3]:
from pathlib import Path  # Importing Path module from pathlib to handle file and directory paths in a cross-platform way.

# Get current working directory
PROJECT_ROOT = Path.cwd()  # Get the current working directory where the notebook is being executed.

# User input for folder location
user_input = input("Enter relative path (default='.'): ").strip() or "."  # Prompt user to input the folder path or use the default value "."
target_path = (PROJECT_ROOT / user_input).resolve()  # Combine the project root directory with user input to get the full target path.

# Create the necessary directories if they don't already exist
target_path.mkdir(parents=True, exist_ok=True)  # Creates the target folder and any necessary parent directories.
print(f"Using path: {target_path}")  # Print out the path being used for confirmation.

# Define paths for saving data and PDFs
data_folder = 'data'  # This folder will store the new Weekly Reports (post-2013), which are in PDF format.
raw_data_subfolder = os.path.join(data_folder, 'raw')  # Subfolder for saving the raw PDFs exactly as downloaded from the BCRP website.
input_data_subfolder = os.path.join(data_folder, 'input')  # Subfolder for saving reduced PDFs that contain only the selected pages with GDP growth tables.
output_data_subfolder = os.path.join(data_folder, 'output')

# Additional folders for metadata, records, and alert tracking
metadata_folder = 'metadata'  # Folder for storing metadata files like wr_metadata.csv.
record_folder = 'record'  # Folder for storing .txt files that track the files already processed to avoid reprocessing them.
debugging_folder = '_debugging'  # Folder for storing .html files that perform debugging.

# Create additional required folders
for folder in [data_folder, raw_data_subfolder, input_data_subfolder, output_data_subfolder, metadata_folder, record_folder, debugging_folder]:
    os.makedirs(folder, exist_ok=True)  # Create the additional folders if they don't exist.
    print(f"📂 {folder} created")  # Print confirmation for each of these additional folders.


Enter relative path (default='.'):  .


Using path: C:\Users\Jason Cruz\OneDrive\Documentos\RA\CIUP\GitHub\FiscalTone
📂 data created
📂 data\raw created
📂 data\input created
📂 data\output created
📂 metadata created
📂 record created
📂 _debugging created


---

## 1. Downloading Textual Data

---

In [14]:
import os
import time
from timeit import default_timer as timer
import requests
import pandas as pd
from urllib.parse import urlparse, parse_qs, unquote
from bs4 import BeautifulSoup


# ======================================================
# Utility: classify PPT / presentation documents
# ======================================================

def is_presentation_pdf(url_or_text):
    if not url_or_text:
        return False
    s = url_or_text.lower()
    ppt_keywords = [
        "ppt", "presentacion", "presentación", "diapositiva",
        "slides", "conferencia", "powerpoint"
    ]
    return any(kw in s for kw in ppt_keywords)


# ======================================================
# Utility: extract all PDF links
# ======================================================

def extract_pdf_links(dsoup):
    pdf_links = []
    for a in dsoup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" in href.lower():
            txt = a.text.strip() if a.text else ""
            pdf_links.append((href, txt))
    return pdf_links


# ======================================================
# Utility: select the best PDF from multiple candidates
# ======================================================

def select_appropriate_pdf(pdf_links):
    if not pdf_links:
        return None

    filtered = [
        (href, txt) for href, txt in pdf_links
        if not is_presentation_pdf(href) and not is_presentation_pdf(txt)
    ]

    candidates = filtered if filtered else pdf_links

    priority_keywords = [
        "comunicado", "informe", "nota", "reporte",
        "documento", "pronunciamiento"
    ]

    def score(x):
        href, _ = x
        h = href.lower()
        return sum(kw in h for kw in priority_keywords)

    return max(candidates, key=score)[0]


# ======================================================
# SCRAPER – Incremental: only visits detail pages not seen before
# ======================================================

def scrape_cf(url, already_scraped_pages):
    t0 = timer()

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    rows = soup.select("table.table tbody tr")
    new_records = []
    list_records = []

    for row in rows:
        try:
            date = row.select_one("td.size100 p").text.strip()
            link_tag = row.select_one("td a")
            title = link_tag.text.strip()
            page_url = link_tag["href"]

            list_records.append({
                "date": date,
                "title": title,
                "page_url": page_url
            })

            if page_url in already_scraped_pages:
                continue

            detail = requests.get(page_url, timeout=15)
            dsoup = BeautifulSoup(detail.text, "html.parser")

            pdf_url = None

            # A) <a> PDF
            pdf_links = extract_pdf_links(dsoup)
            if pdf_links:
                pdf_url = select_appropriate_pdf(pdf_links)

            # B) iframe
            if not pdf_url:
                iframe = dsoup.find("iframe", src=lambda x: x and ".pdf" in x.lower())
                if iframe:
                    src = iframe["src"]
                    if src.startswith("//"):
                        src = "https:" + src
                    pdf_url = src

            # C) Google Docs viewer
            if not pdf_url:
                iframe = dsoup.find("iframe", src=lambda x: x and "docs.google.com" in x.lower())
                if iframe:
                    parsed = urlparse(iframe["src"])
                    q = parse_qs(parsed.query)
                    if "url" in q:
                        pdf_url = unquote(q["url"][0])

            # D) PDF viewer "Guardar"
            if not pdf_url:
                button = dsoup.find("button", id="downloadButton") or dsoup.find("span", string="Guardar")
                if button:
                    parent = button.find_parent("a")
                    if parent and parent.has_attr("href"):
                        pdf_url = parent["href"]

            filename = pdf_url.split("/")[-1] if pdf_url else None

            new_records.append({
                "date": date,
                "title": title,
                "page_url": page_url,
                "final_pdf_url": pdf_url,
                "final_pdf_filename": filename
            })

        except Exception as e:
            print(f"❌ Error processing row: {e}")

    print(f"⌛ scrape_cf_expanded executed in {timer() - t0:.2f} sec")
    return list_records, new_records



# ======================================================
# DOWNLOADER — with incremental CSV-safe saving (even if interrupted)
# ======================================================

def pdf_downloader(cf_urls, raw_pdf_folder, metadata_folder, metadata_csv):

    t0 = timer()
    os.makedirs(raw_pdf_folder, exist_ok=True)
    os.makedirs(metadata_folder, exist_ok=True)

    metadata_path = os.path.join(metadata_folder, f"{metadata_csv}.csv")

    # Load previous metadata safely
    if os.path.exists(metadata_path):
        old_df = pd.read_csv(metadata_path, dtype=str)
        old_urls = set(old_df["final_pdf_url"].dropna())
        old_pages = set(old_df["page_url"].dropna())
    else:
        old_df = pd.DataFrame()
        old_urls = set()
        old_pages = set()

    all_new_records = []

    # SCRAPE incremental
    for url in cf_urls:
        print(f"\n🌐 Scraping list page: {url}")
        list_records, new_page_records = scrape_cf(
            url, already_scraped_pages=old_pages
        )
        all_new_records.extend(new_page_records)

    if not all_new_records:
        print("\n🔎 No new pages: skipping download.")
        print(f"📝 Metadata unchanged: {metadata_path}")
        # → return existing CSV as DataFrame
        return pd.read_csv(metadata_path, dtype=str)

    new_df = pd.DataFrame(all_new_records).dropna(subset=["final_pdf_url"])
    mask_new = ~new_df["final_pdf_url"].isin(old_urls)
    df_to_download = new_df[mask_new].copy()

    # Sort oldest → newest
    df_to_download["date"] = pd.to_datetime(df_to_download["date"], dayfirst=True)
    df_to_download = df_to_download.sort_values("date").reset_index(drop=True)

    print(f"\n🔎 Found {len(df_to_download)} new PDFs to download")

    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/pdf"}

    # ---- Incremental download + incremental CSV writing ----
    temp_df = old_df.copy()

    for i, row in df_to_download.iterrows():
        pdf_url = row["final_pdf_url"]
        filename = row["final_pdf_filename"]
        page_url = row["page_url"]
        filepath = os.path.join(raw_pdf_folder, filename)

        print(f"\n[{i+1}/{len(df_to_download)}] 📄 {filename}")
        print(f"🔗 {pdf_url}")
        
        success = False

        # Primary download attempt
        try:
            r = requests.get(pdf_url, headers=headers, timeout=20)
            r.raise_for_status()
            with open(filepath, "wb") as f:
                f.write(r.content)
            print(f"✅ Saved {filename}")
            success = True

        except Exception as e:
            print(f"⚠️ Primary failed: {e}")
        
        # Fallback attempt
        if not success:
            try:
                print("🔁 Trying extended fallback…")
                detail = requests.get(page_url, timeout=15)
                dsoup = BeautifulSoup(detail.text, "html.parser")
        
                iframe_url = None
        
                # 1) <embed src="...pdf">
                embed = dsoup.find("embed", src=lambda x: x and ".pdf" in x.lower())
                if embed:
                    iframe_url = embed["src"]
        
                # 2) <div data-pdf-src="...pdf">
                if not iframe_url:
                    divpdf = dsoup.find("div", attrs={"data-pdf-src": True})
                    if divpdf and ".pdf" in divpdf["data-pdf-src"].lower():
                        iframe_url = divpdf["data-pdf-src"]
        
                # Normalize URL
                if iframe_url and iframe_url.startswith("//"):
                    iframe_url = "https:" + iframe_url
        
                if iframe_url:
                    print(f"   ⇢ fallback PDF URL: {iframe_url}")
        
                    r2 = requests.get(iframe_url, headers=headers, timeout=20)
                    r2.raise_for_status()
        
                    if "pdf" not in r2.headers.get("Content-Type", "").lower():
                        raise ValueError("Server returned HTML instead of PDF")
        
                    with open(filepath, "wb") as f:
                        f.write(r2.content)
        
                    print(f"✅ Saved via embed/data-pdf-src fallback: {filename}")
                    success = True
                else:
                    print("❌ No embed/data-pdf-src found")
        
            except Exception as e2:
                print(f"❌ Extended fallback failed: {e2}")

        # === Incremental update to metadata (even if interrupted later) ===
        temp_df = pd.concat([temp_df, pd.DataFrame([row])], ignore_index=True)
        temp_df.to_csv(metadata_path, index=False)

        # avoid rate limit
        time.sleep(1)

    print("\n📊 Summary:")
    print(f"📝 Metadata saved incrementally: {metadata_path}")
    print(f"⏱️ Done in {round(timer() - t0, 2)} sec")

    # === RETURN THE CSV AS DATAFRAME ===
    final_df = pd.read_csv(metadata_path, dtype=str)
    return final_df


In [17]:
cf_urls = [
    "https://cf.gob.pe/p/informes/",
    "https://cf.gob.pe/p/comunicados/"
]

metadata_df = pdf_downloader(
    cf_urls=cf_urls,
    raw_pdf_folder=raw_data_subfolder,
    metadata_folder=metadata_folder,
    metadata_csv="cf_metadata"
)


🌐 Scraping list page: https://cf.gob.pe/p/informes/
⌛ scrape_cf_expanded executed in 52.03 sec

🌐 Scraping list page: https://cf.gob.pe/p/comunicados/
⌛ scrape_cf_expanded executed in 25.52 sec

🔎 Found 79 new PDFs to download

[1/79] 📄 Informe_CF_N_001-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_001-2016.pdf
✅ Saved Informe_CF_N_001-2016.pdf

[2/79] 📄 Informe_CF_N_002-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_002-2016.pdf
✅ Saved Informe_CF_N_002-2016.pdf

[3/79] 📄 Informe_CF_N_003-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_003-2016.pdf
✅ Saved Informe_CF_N_003-2016.pdf

[4/79] 📄 Informe_CF_N_005-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_005-2016.pdf
✅ Saved Informe_CF_N_005-2016.pdf

[5/79] 📄 Informe_CF_N_004-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_004-2016.pdf
✅ Saved Informe_CF_N_004-2016.pdf

[6/79] 📄 Informe_CF_N_006-2016.pdf
🔗 https://cf.gob.pe/pub/Informe_CF_N_006-2016.pdf
✅ Saved Informe_CF_N_006-2016.pdf

[7/79] 📄 Informe_CF_N_007-2016.pdf
🔗 https://cf.gob

In [18]:
metadata_df.head(10)

,date,title,page_url,final_pdf_url,final_pdf_filename
0,2016-01-28,Informe CF N° 001-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_001-2016.pdf,Informe_CF_N_001-2016.pdf
1,2016-04-15,Informe CF N° 002-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_002-2016.pdf,Informe_CF_N_002-2016.pdf
2,2016-06-23,Informe CF N° 003-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_003-2016.pdf,Informe_CF_N_003-2016.pdf
3,2016-08-18,Informe CF N° 005-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_005-2016.pdf,Informe_CF_N_005-2016.pdf
4,2016-08-18,Informe CF N° 004-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_004-2016.pdf,Informe_CF_N_004-2016.pdf
5,2016-08-26,Informe CF N° 006-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_006-2016.pdf,Informe_CF_N_006-2016.pdf
6,2016-10-04,Informe CF N° 007-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_007-2016.pdf,Informe_CF_N_007-2016.pdf
7,2016-12-26,Informe CF N° 008-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_008-2016.pdf,Informe_CF_N_008-2016.pdf
8,2017-05-03,Informe CF N° 001-2017 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/INFORME_N_001-2017-CF.pdf,INFORME_N_001-2017-CF.pdf
9,2017-05-09,Informe CF N° 002-2017 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/INFORME_N_002-2017-CF.pdf,INFORME_N_002-2017-CF.pdf


### Delete selected documents

In [19]:
# Lista de archivos a eliminar
pdfs_to_remove = [
    "Informe-anual-2017_CF_vf.pdf", # This is a way too long document containing statistical analaysis. We're focus only on text as comunicados from CF.
    "Informe-anual-del-Consejo-Fiscal-2018-version-final1.pdf"
]


In [20]:
def remove_unwanted_pdfs(folder_path, filenames_to_remove):
    """
    Deletes specific unwanted PDF files from a given folder.

    Parameters:
    - folder_path: str, the directory containing the PDFs
    - filenames_to_remove: list of str, filenames to delete
    """
    t0 = timer()
    removed_count = 0

    print(f"🧹 Removing in: {folder_path}")
    for filename in filenames_to_remove:
        full_path = os.path.join(folder_path, filename)
        if os.path.exists(full_path):
            os.remove(full_path)
            print(f"🗑️ Deleted: {filename}")
            removed_count += 1
        else:
            print(f"⚠️ File not found: {filename}")

    t1 = timer()

    print("\n📊 Summary:")

    print(f"\n🧹 Cleanup complete. Total files removed: {removed_count}")
    print(f"⏱️ Time taken: {t1 - t0:.2f} seconds")



In [21]:
remove_unwanted_pdfs(raw_data_subfolder, pdfs_to_remove)

🧹 Removing in: data\raw
🗑️ Deleted: Informe-anual-2017_CF_vf.pdf
🗑️ Deleted: Informe-anual-del-Consejo-Fiscal-2018-version-final1.pdf

📊 Summary:

🧹 Cleanup complete. Total files removed: 2
⏱️ Time taken: 0.00 seconds


# Clasificando entre scanned and editable PDFs

Please, compruebe que la carpeta "" contenga scaneados y que la carpeta "" ediatables. En caso de que no se haya clasificado correctamente los pdfs, agreguelos manualemnte. Esto es vital para los próximos códigos.

In [22]:
import os
import fitz  # PyMuPDF, used to extract text from PDFs
import shutil
import pandas as pd
from timeit import default_timer as timer

def is_editable_pdf(file_path, min_text_length=20):
    """Check if a PDF contains extractable text (editable)."""
    try:
        with fitz.open(file_path) as doc:
            return len("".join(page.get_text() for page in doc).strip()) >= min_text_length
    except Exception as e:
        print(f"❌ Error reading {file_path}: {e}")
        return False


def classify_pdfs_by_type(classification_folder, metadata_folder, metadata_csv="cf_metadata"):
    """
    Classifies PDF files from a given directory into 'editable' and 'scanned' subfolders, 
    and updates a metadata CSV file with the type of each document (editable or scanned).

    Parameters:
    - classification_folder: str, the directory where 'editable' and 'scanned' subfolders will be created.
    - metadata_folder: str, the directory where the metadata CSV file is located.
    - metadata_csv: str, the name of the CSV file containing metadata (without the '.csv' extension).
    """
    # Ensure classification_folder is a list, even if a single folder is passed
    if isinstance(classification_folder, str):
        classification_folder = [classification_folder]

    # Create the 'editable' and 'scanned' subfolders within the classification_folder
    output_dir_editable = os.path.join(classification_folder[0], "editable")
    output_dir_scanned = os.path.join(classification_folder[0], "scanned")
    os.makedirs(output_dir_editable, exist_ok=True)
    os.makedirs(output_dir_scanned, exist_ok=True)

    total_files = 0
    scanned_count = 0
    editable_count = 0

    # Add .csv extension to metadata_csv if not provided
    metadata_csv_path = os.path.join(metadata_folder, f"{metadata_csv}.csv")

    # Load the metadata CSV file
    metadata_df = pd.read_csv(metadata_csv_path)

    # Add a new "type" column if it doesn't exist
    if 'type' not in metadata_df.columns:
        metadata_df['type'] = ''

    t0 = timer()
    print("🔍 Starting PDF classification...\n")

    # Iterate through the provided folders
    for folder in classification_folder:
        for filename in os.listdir(folder):
            if filename.lower().endswith(".pdf"):
                total_files += 1
                pdf_path = os.path.join(folder, filename)

                # Classify and move the PDF to the appropriate folder
                if is_editable_pdf(pdf_path):
                    shutil.move(pdf_path, os.path.join(output_dir_editable, filename))
                    editable_count += 1
                    file_type = "editable"
                else:
                    shutil.move(pdf_path, os.path.join(output_dir_scanned, filename))
                    scanned_count += 1
                    file_type = "scanned"

                # Update the metadata CSV with the file type
                metadata_df.loc[metadata_df['final_pdf_filename'] == filename, 'type'] = file_type

    # Save the updated CSV back to the metadata folder
    metadata_df.to_csv(metadata_csv_path, index=False)

    t1 = timer()

    # Print a summary
    print("\n📊 Summary:")
    print(f"📄 Total PDFs processed: {total_files}")
    print(f"💻 Editable PDFs: {editable_count}")
    print(f"🖨️ Scanned PDFs: {scanned_count}")
    print(f"📁 Saved editable PDFs in: '{output_dir_editable}'")
    print(f"📁 Saved scanned PDFs in: '{output_dir_scanned}'")
    print(f"⏱️ Time taken: {t1 - t0:.2f} seconds")
    print(f"📑 Metadata CSV updated: '{metadata_csv_path}'")

    # Return the updated CSV as a DataFrame for inspection
    return metadata_df



In [23]:
# Define the folder paths
classification_folder = raw_data_subfolder
metadata_folder = metadata_folder
metadata_csv = "cf_metadata"  # Note: without the ".csv" extension

# Call the function to classify PDFs, update the CSV, and return the updated DataFrame
updated_metadata_df = classify_pdfs_by_type(classification_folder, metadata_folder, metadata_csv)

🔍 Starting PDF classification...


📊 Summary:
📄 Total PDFs processed: 77
💻 Editable PDFs: 64
🖨️ Scanned PDFs: 13
📁 Saved editable PDFs in: 'data\raw\editable'
📁 Saved scanned PDFs in: 'data\raw\scanned'
⏱️ Time taken: 2.24 seconds
📑 Metadata CSV updated: 'metadata\cf_metadata.csv'


In [24]:
# Inspect the updated DataFrame in Jupyter Notebook or JupyterLab
updated_metadata_df.head()  # Display the first few rows of the updated CSV

,date,title,page_url,final_pdf_url,final_pdf_filename,type
0,2016-01-28,Informe CF N° 001-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_001-2016.pdf,Informe_CF_N_001-2016.pdf,scanned
1,2016-04-15,Informe CF N° 002-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_002-2016.pdf,Informe_CF_N_002-2016.pdf,scanned
2,2016-06-23,Informe CF N° 003-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_003-2016.pdf,Informe_CF_N_003-2016.pdf,scanned
3,2016-08-18,Informe CF N° 005-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_005-2016.pdf,Informe_CF_N_005-2016.pdf,scanned
4,2016-08-18,Informe CF N° 004-2016 – Opinión del Consejo F...,https://cf.gob.pe/documentos/informes/informe-...,https://cf.gob.pe/pub/Informe_CF_N_004-2016.pdf,Informe_CF_N_004-2016.pdf,scanned


In [ ]:
def combine_metadata(csv_reports="cf_reports.csv", csv_announcements="cf_announcements.csv", output_csv="cf_pdfs_metadata.csv"):
    """
    Combines metadata from two separate CSV files (reports and announcements) into one.

    Parameters:
    - csv_reports: str, path to the reports metadata CSV
    - csv_announcements: str, path to the announcements metadata CSV
    - output_csv: str, path for the combined output CSV

    Saves:
    - A single CSV file containing all metadata under the name specified in output_csv
    """
    t0 = timer()
    print("📥 Loading metadata...")

    df_reports = pd.read_csv(csv_reports)
    df_announcements = pd.read_csv(csv_announcements)

    df_combined = pd.concat([df_reports, df_announcements], ignore_index=True)
    df_combined.to_csv(output_csv, index=False)

    t1 = timer()
    print(f"✅ Combined metadata saved as '{output_csv}'")
    print(f"🧾 Total records: {len(df_combined)}")
    print(f"⏱️ Time taken: {t1 - t0:.2f} seconds")


In [ ]:
combine_metadata()

<div id="1-3"; style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #ff8575;">
        1.3 Split into paragraphs
    </h2>
</div>

----

## 3. Text Preprocessing

---

# Editable PDFs

In [ ]:
def process_editable_pdfs(folder_path, metadata_csv_path="cf_pdfs_metadata.csv", output_csv="raw_editable_pdfs_dataset.csv"):
    """
    Processes editable PDF documents, extracting structured paragraphs and saving to CSV.

    Parameters:
    - folder_path: str, path to the folder containing editable PDFs
    - metadata_csv_path: str, path to the CSV file with document metadata
    - output_csv: str, output CSV filename to save extracted paragraphs

    Returns:
    - DataFrame with extracted structured text and metadata
    """
    t0 = timer()
    print("📥 Loading metadata...")
    metadata_df = pd.read_csv(metadata_csv_path)

    # --- Extract doc_type, doc_id, year from title ---
    def extract_doc_info(row):
        title = row["title"]
        match = re.search(r"\b(Informe|Comunicado)\b(?:\s+CF)?(?:\s+(?:N[°ºo]|No))?\s*(\d{2,4})", title, re.IGNORECASE)
        doc_type = match.group(1).capitalize() if match else None
        doc_id = match.group(2) if match and match.lastindex >= 2 else None
        year_match = re.search(r"\b(\d{4})\b", str(row.get("date", "")))
        year = year_match.group(1) if year_match else None
        return pd.Series({"doc_type": doc_type, "doc_id": doc_id, "year": year})

    metadata_df[["doc_type", "doc_id", "year"]] = metadata_df.apply(extract_doc_info, axis=1)

    print("🧠 Metadata enriched. Starting paragraph extraction...\n")
    all_records = []
    filenames = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]
    total_files = len(filenames)

    for idx, filename in enumerate(sorted(filenames), start=1):
        file_path = os.path.join(folder_path, filename)

        try:
            print(f"📄 Processing: {idx}. {filename}")
            with pdfplumber.open(file_path) as pdf:
                paragraph_counter = 1
                anexo_found = False

                for page_num, page in enumerate(pdf.pages, start=1):
                    if anexo_found:
                        break

                    # Extract words with their size and vertical position
                    words = page.extract_words(extra_attrs=["size", "top"])
                    FONT_MIN = 11.0
                    FONT_MAX = 11.9
                    clean_words = [w for w in words if FONT_MIN <= w["size"] <= FONT_MAX]
                    if not clean_words:
                        continue

                    # Group words by their vertical position to form lines
                    lines_dict = {}
                    for word in clean_words:
                        line_top = round(word["top"], 1)
                        lines_dict.setdefault(line_top, []).append(word["text"])

                    lines = [
                        " ".join(words).strip()
                        for _, words in sorted(lines_dict.items())
                        if words
                    ]

                    if not lines:
                        continue

                    page_text = "\n".join(lines)

                    # 🚫 Stop extraction at "Anexo"
                    match = re.search(r"(?mi)^ *Anexos?\b[\s\w]*:?", page_text)
                    if match:
                        page_text = page_text[:match.start()].strip()
                        anexo_found = True
                        print(f"🛑 'Anexo' detected on page {page_num}. Truncating content.")

                    # Paragraph segmentation
                    lines = page_text.strip().split("\n")
                    lines = [line.strip() for line in lines if line.strip()]
                    paragraph_lines = []

                    for i, line in enumerate(lines):
                        is_new_paragraph = (
                            line.startswith("•")
                            or line.startswith("➢")
                            or (i > 0 and lines[i - 1].strip().endswith("."))
                            or (i > 0 and len(lines[i - 1].split()) <= 3)
                        )

                        if is_new_paragraph:
                            if paragraph_lines:
                                all_records.append({
                                    "filename": filename,
                                    "page": page_num,
                                    "paragraph_id": paragraph_counter,
                                    "text": " ".join(paragraph_lines).strip()
                                })
                                paragraph_counter += 1
                            paragraph_lines = [line]
                        else:
                            paragraph_lines.append(line)

                    if paragraph_lines:
                        all_records.append({
                            "filename": filename,
                            "page": page_num,
                            "paragraph_id": paragraph_counter,
                            "text": " ".join(paragraph_lines).strip()
                        })
                        paragraph_counter += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

    df = pd.DataFrame(all_records)
    if df.empty:
        print("⚠️ No text extracted.")
        return df

    # Merge with metadata
    df = df.merge(
        metadata_df[["pdf_filename", "title", "doc_type", "doc_id", "year", "date"]],
        left_on="filename", right_on="pdf_filename", how="left"
    )

    df = df[["title", "doc_type", "doc_id", "year", "date", "page", "paragraph_id", "text"]]
    df.to_csv(output_csv, index=False)

    t1 = timer()
    print(f"\n✅ Extraction complete. Total paragraphs: {len(df)}")
    print(f"💾 Saved to: {output_csv}")
    print(f"⏱️ Time taken: {t1 - t0:.2f} seconds")
    return df

In [ ]:
df_editable = process_editable_pdfs("editable_pdf")

In [ ]:
df_editable

# Scanned PDFs

"""
Procesa PDFs escaneados usando OCR para extraer párrafos por página y metadatos.

Args:
    folder_path (str): Ruta a la carpeta con archivos PDF escaneados.
    dpi (int): Resolución al convertir PDF a imagen.
    lang (str): Idioma para el OCR ('spa' para español, 'eng' para inglés, etc.).

Returns:
    pd.DataFrame: DataFrame con columnas:
        ['filename', 'year', 'date', 'announcement', 'page', 'paragraph_id', 'text']
"""    

Puede tardar un poco más. Se excluye pies de pagina con detección visual de lineas que marcan el inicio. Se páginas de anexos.

In [ ]:
# === OCR Utilities ===

def detect_cut_line_y(image, min_length_ratio=0.2, y_range=(0.5, 0.85), debug_path=None):
    """
    Detects a horizontal line that likely marks the beginning of the footer in a scanned PDF page.
    
    Parameters:
        image (PIL.Image): Page image to analyze.
        min_length_ratio (float): Minimum line length relative to image width.
        y_range (tuple): Vertical range (as a proportion of image height) where footer lines are expected.
        debug_path (str): Optional path to save debug image with the detected line.
    
    Returns:
        int: Y-coordinate to crop the image above the footer line, or image height if no line is found.
    """
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)

    lines = cv2.HoughLinesP(
        thresh, 1, np.pi / 180, threshold=100,
        minLineLength=int(image.width * min_length_ratio), maxLineGap=5
    )

    if lines is not None:
        height = image.height
        min_y, max_y = int(height * y_range[0]), int(height * y_range[1])

        horizontal_lines = [
            (x1, y1, x2, y2) for x1, y1, x2, y2 in lines[:, 0]
            if abs(y1 - y2) <= 3 and min_y <= y1 <= max_y
        ]

        if horizontal_lines:
            best_line = min(horizontal_lines, key=lambda l: l[1])
            if debug_path:
                img_dbg = image.copy()
                dbg_np = np.array(img_dbg)
                cv2.line(dbg_np, (best_line[0], best_line[1]), (best_line[2], best_line[3]), (0, 0, 255), 2)
                cv2.imwrite(debug_path, cv2.cvtColor(dbg_np, cv2.COLOR_RGB2BGR))
            return best_line[1]

    return image.height  # Default: no footer line detected


# OCR

# Major function

In [ ]:
# === OCR Utilities ===

def detect_cut_line_y(image, min_length_ratio=0.17, y_range=(0.55, 0.90), debug_path=None):
    """
    Detects a horizontal line likely indicating the beginning of the footer in scanned PDFs.

    Args:
        image (PIL.Image): Page image.
        min_length_ratio (float): Minimum length of line relative to image width.
        y_range (tuple): Vertical range to search (proportional to height).
        debug_path (str): Optional file path to save a debug image with detected line.

    Returns:
        int: Y-coordinate of the detected line, or image height if none found.
    """
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 160, 255, cv2.THRESH_BINARY_INV)

    lines = cv2.HoughLinesP(
        thresh, 1, np.pi / 180, threshold=80,
        minLineLength=int(image.width * min_length_ratio), maxLineGap=5
    )

    if lines is not None:
        height = image.height
        min_y, max_y = int(height * y_range[0]), int(height * y_range[1])

        horizontal_lines = [
            (x1, y1, x2, y2) for x1, y1, x2, y2 in lines[:, 0]
            if abs(y1 - y2) <= 5 and min_y <= y1 <= max_y
        ]

        if horizontal_lines:
            best_line = min(horizontal_lines, key=lambda l: l[1])
            if debug_path:
                img_dbg = image.copy()
                dbg_np = np.array(img_dbg)
                cv2.line(dbg_np, (best_line[0], best_line[1]), (best_line[2], best_line[3]), (0, 0, 255), 2)
                cv2.imwrite(debug_path, cv2.cvtColor(dbg_np, cv2.COLOR_RGB2BGR))
            return best_line[1]

    return image.height  # No line detected → return full height


In [ ]:
# === OCR Processing for Scanned PDFs ===

def process_scanned_pdfs(folder_path, metadata_csv_path="cf_pdfs_metadata.csv", dpi=300, lang='spa', debug=True):
    """
    Extracts paragraphs from scanned PDFs using OCR, excluding footers and annexes.

    Args:
        folder_path (str): Folder with scanned PDFs.
        metadata_csv_path (str): Path to metadata CSV file.
        dpi (int): Resolution used to convert PDFs to images.
        lang (str): OCR language code.
        debug (bool): Whether to save debug images with detected lines.

    Returns:
        pd.DataFrame: Paragraph-level extracted data.
    """
    print("📥 Loading metadata...")
    metadata_df = pd.read_csv(metadata_csv_path)

    def extract_doc_info(row):
        title = row.get("title", "")
        match = re.search(r"\b(Informe|Comunicado)\b(?:\s+CF)?(?:\s+(?:N[°ºo]|No))?\s*(\d{2,4})", title, re.IGNORECASE)
        doc_type = match.group(1).capitalize() if match else None
        doc_id = match.group(2) if match and match.lastindex >= 2 else None
        year_match = re.search(r"\b(\d{4})\b", str(row.get("date", "")))
        year = year_match.group(1) if year_match else None
        return pd.Series({"doc_type": doc_type, "doc_id": doc_id, "year": year})

    metadata_df[["doc_type", "doc_id", "year"]] = metadata_df.apply(extract_doc_info, axis=1)
    print("🧠 Metadata enriched. Starting OCR paragraph extraction...\n")

    all_records = []
    filenames = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]
    total_files = len(filenames)

    for idx, filename in enumerate(sorted(filenames), start=1):
        file_path = os.path.join(folder_path, filename)
        print(f"🖨️ OCR Processing ({idx}/{total_files}): {filename}")
        try:
            images = convert_from_path(file_path, dpi=dpi)
            paragraph_counter = 1
            anexo_found = False

            for page_num, image in enumerate(images, start=1):
                if anexo_found:
                    break

                debug_path = None
                if debug:
                    os.makedirs("debug_lines", exist_ok=True)
                    debug_path = f"debug_lines/{filename}_page_{page_num}.png"

                cut_y = detect_cut_line_y(image, debug_path=debug_path)
                cropped_img = image.crop((0, 0, image.width, cut_y))

                page_text = pytesseract.image_to_string(cropped_img, lang=lang)

                if not page_text.strip():
                    continue

                # Stop at 'Anexo'
                match = re.search(r"(?mi)^ *Anexos?\b[\s\w]*:?", page_text)
                if match:
                    page_text = page_text[:match.start()].strip()
                    anexo_found = True
                    print(f"🛑 'Anexo' detected on page {page_num}. Truncating content.")

                lines = [line.strip() for line in page_text.split("\n") if line.strip()]
                paragraph_lines = []

                for i, line in enumerate(lines):
                    is_new_paragraph = (
                        line.startswith("•") or line.startswith("➢") or
                        (i > 0 and lines[i - 1].strip().endswith(".")) or
                        (i > 0 and len(lines[i - 1].split()) <= 3)
                    )
                    if is_new_paragraph:
                        if paragraph_lines:
                            all_records.append({
                                "filename": filename,
                                "page": page_num,
                                "paragraph_id": paragraph_counter,
                                "text": " ".join(paragraph_lines).strip()
                            })
                            paragraph_counter += 1
                        paragraph_lines = [line]
                    else:
                        paragraph_lines.append(line)

                if paragraph_lines:
                    all_records.append({
                        "filename": filename,
                        "page": page_num,
                        "paragraph_id": paragraph_counter,
                        "text": " ".join(paragraph_lines).strip()
                    })
                    paragraph_counter += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

    df = pd.DataFrame(all_records)
    if df.empty:
        print("⚠️ No text extracted.")
        return df

    df = df.merge(
        metadata_df[["pdf_filename", "title", "doc_type", "doc_id", "year", "date"]],
        left_on="filename", right_on="pdf_filename", how="left"
    )
    df = df[["title", "doc_type", "doc_id", "year", "date", "page", "paragraph_id", "text"]]
    print(f"\n✅ OCR extraction complete. Total paragraphs: {len(df)}")
    return df

In [ ]:
# === Run and Save ===
df_scanned = process_scanned_pdfs(
    folder_path="scanned_pdf",
    metadata_csv_path="cf_pdfs_metadata.csv",
    dpi=400,
    lang='spa',
    debug=True
)
df_scanned.to_csv("raw_scanned_pdfs_dataset.csv", index=False)
print("💾 Saved as 'raw_scanned_pdfs_dataset.csv'")

In [ ]:
df_scanned

# NUEVA UTILIDAD

In [ ]:
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from sklearn.cluster import DBSCAN

def filter_text_body_dynamic(
    image,
    size_threshold_percentile=40,
    merge_dist=20,
    expand_margin=10,
    debug_path=None,
    audit_csv_path=None
):
    """
    Filtra regiones de texto fuera del cuerpo principal del documento mediante detección dinámica.

    Args:
        image (PIL.Image): Imagen de entrada.
        size_threshold_percentile (int): Percentil mínimo de altura para considerar caja válida.
        merge_dist (int): Distancia máxima entre cajas para considerar agrupación.
        expand_margin (int): Expansión del marco del cuerpo principal.
        debug_path (str): Ruta para guardar imagen de depuración.
        audit_csv_path (str): Ruta para guardar CSV con info de cajas.

    Returns:
        PIL.Image: Imagen filtrada.
    """
    img_rgb = np.array(image.convert("RGB"))
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(c) for c in contours]
    if not boxes:
        return image

    heights = [h for (_, _, _, h) in boxes]
    height_threshold = np.percentile(heights, size_threshold_percentile)

    filtered_boxes = [(x, y, w, h) for (x, y, w, h) in boxes if h >= height_threshold]
    if not filtered_boxes:
        return image

    centers = np.array([[x + w//2, y + h//2] for (x, y, w, h) in filtered_boxes])
    clustering = DBSCAN(eps=merge_dist, min_samples=3).fit(centers)
    labels = clustering.labels_

    largest_label = pd.Series(labels).value_counts().idxmax()
    main_group_boxes = [box for i, box in enumerate(filtered_boxes) if labels[i] == largest_label]

    x_min = max(0, min(x for x, _, _, _ in main_group_boxes) - expand_margin)
    x_max = max(x + w for x, _, w, _ in main_group_boxes) + expand_margin
    y_min = max(0, min(y for _, y, _, _ in main_group_boxes) - expand_margin)
    y_max = max(y + h for _, y, _, h in main_group_boxes) + expand_margin

    mask = np.zeros_like(gray)
    accepted, rejected = [], []

    for x, y, w, h in boxes:
        cx, cy = x + w // 2, y + h // 2
        if x_min <= cx <= x_max and y_min <= cy <= y_max:
            accepted.append((x, y, w, h))
            cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
        else:
            rejected.append((x, y, w, h))

    result = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)
    filtered_image = Image.fromarray(result)

    if debug_path:
        debug_img = img_rgb.copy()
        for x, y, w, h in accepted:
            cv2.rectangle(debug_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        for x, y, w, h in rejected:
            cv2.rectangle(debug_img, (x, y), (x + w, y + h), (0, 0, 255), 1)
        cv2.rectangle(debug_img, (x_min, y_min), (x_max, y_max), (255, 0, 255), 2)
        cv2.imwrite(debug_path, cv2.cvtColor(debug_img, cv2.COLOR_RGB2BGR))

    if audit_csv_path:
        records = []
        for x, y, w, h in accepted:
            records.append({"x": x, "y": y, "w": w, "h": h, "status": "accepted"})
        for x, y, w, h in rejected:
            records.append({"x": x, "y": y, "w": w, "h": h, "status": "rejected"})
        pd.DataFrame(records).to_csv(audit_csv_path, index=False)

    return filtered_image


In [ ]:
import os
import re
import pytesseract
import pandas as pd
from pdf2image import convert_from_path
from PIL import Image

def process_scanned_pdfs(folder_path, metadata_csv_path="cf_pdfs_metadata.csv", dpi=300, lang='spa', debug=True):
    print("📥 Loading metadata...")
    metadata_df = pd.read_csv(metadata_csv_path)

    def extract_doc_info(row):
        title = row.get("title", "")
        match = re.search(r"\b(Informe|Comunicado)\b(?:\s+CF)?(?:\s+(?:N[°ºo]|No))?\s*(\d{2,4})", title, re.IGNORECASE)
        doc_type = match.group(1).capitalize() if match else None
        doc_id = match.group(2) if match and match.lastindex >= 2 else None
        year_match = re.search(r"\b(\d{4})\b", str(row.get("date", "")))
        year = year_match.group(1) if year_match else None
        return pd.Series({"doc_type": doc_type, "doc_id": doc_id, "year": year})

    metadata_df[["doc_type", "doc_id", "year"]] = metadata_df.apply(extract_doc_info, axis=1)
    print("🧠 Metadata enriched. Starting OCR paragraph extraction...\n")

    all_records = []
    filenames = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]
    total_files = len(filenames)

    for idx, filename in enumerate(sorted(filenames), start=1):
        file_path = os.path.join(folder_path, filename)
        print(f"🖨️ OCR Processing ({idx}/{total_files}): {filename}")
        try:
            images = convert_from_path(file_path, dpi=dpi)
            paragraph_counter = 1
            anexo_found = False

            for page_num, image in enumerate(images, start=1):
                if anexo_found:
                    break

                os.makedirs("debug_lines", exist_ok=True)
                debug_path = f"debug_lines/{filename}_page_{page_num}.png" if debug else None
                audit_csv_path = f"debug_lines/{filename}_page_{page_num}_boxes.csv" if debug else None

                # 🧠 NUEVA función dinámica
                filtered_img = filter_text_body_dynamic(
                    image,
                    size_threshold_percentile=40,
                    merge_dist=20,
                    expand_margin=10,
                    debug_path=debug_path,
                    audit_csv_path=audit_csv_path
                )

                page_text = pytesseract.image_to_string(filtered_img, lang=lang)
                if not page_text.strip():
                    continue

                # 🛑 Stop at Anexos
                match = re.search(r"(?mi)^ *Anexos?\b[\s\w]*:?", page_text)
                if match:
                    page_text = page_text[:match.start()].strip()
                    anexo_found = True
                    print(f"🛑 'Anexo' detected on page {page_num}. Truncating content.")

                lines = [line.strip() for line in page_text.split("\n") if line.strip()]
                paragraph_lines = []

                for i, line in enumerate(lines):
                    is_new_paragraph = (
                        line.startswith("•") or line.startswith("➢") or
                        (i > 0 and lines[i - 1].strip().endswith(".")) or
                        (i > 0 and len(lines[i - 1].split()) <= 3)
                    )
                    if is_new_paragraph:
                        if paragraph_lines:
                            all_records.append({
                                "filename": filename,
                                "page": page_num,
                                "paragraph_id": paragraph_counter,
                                "text": " ".join(paragraph_lines).strip()
                            })
                            paragraph_counter += 1
                        paragraph_lines = [line]
                    else:
                        paragraph_lines.append(line)

                if paragraph_lines:
                    all_records.append({
                        "filename": filename,
                        "page": page_num,
                        "paragraph_id": paragraph_counter,
                        "text": " ".join(paragraph_lines).strip()
                    })
                    paragraph_counter += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

    df = pd.DataFrame(all_records)
    if df.empty:
        print("⚠️ No text extracted.")
        return df

    df = df.merge(
        metadata_df[["pdf_filename", "title", "doc_type", "doc_id", "year", "date"]],
        left_on="filename", right_on="pdf_filename", how="left"
    )
    df = df[["title", "doc_type", "doc_id", "year", "date", "page", "paragraph_id", "text"]]
    print(f"\n✅ OCR extraction complete. Total paragraphs: {len(df)}")
    return df


In [ ]:
df_scanned = process_scanned_pdfs(
    folder_path="sc",  # Cambia por tu carpeta de PDFs
    metadata_csv_path="cf_pdfs_metadata.csv",
    dpi=400,
    lang='spa',
    debug=True
)
df_scanned.to_csv("raw_scanned_pdfs_dataset.csv", index=False)
print("💾 Saved as 'raw_scanned_pdfs_dataset.csv'")


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: #033280; font-weight: bold;">
        <a href="#outilne" style="color: #033280; text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: #cd301b; text-decoration: none;">Back to the outline.</a>
</div>

# Concatenate

In [ ]:
import pandas as pd

# Concatenar los dos dataframes
text_df = pd.concat([df_editable, df_scanned], ignore_index=True)

# Ordenar por la columna 'date'
text_df = text_df.sort_values(by='date')

# Si deseas resetear los índices después de la concatenación
text_df = text_df.reset_index(drop=True)

In [ ]:
# Exportar a CSV con el mismo nombre
text_df.to_csv("raw_text_dataset.csv", index=False, encoding="utf-8")

In [ ]:
text_df

In [ ]:
text_df.count()

In [ ]:
# Filtrar las filas donde "text" empieza con una letra minúscula
text_df[text_df["text"].str.match(r"^[a-z]")]

<div id="1-4"; style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #ff8575;">
        1.4 Noise reduction
    </h2>
</div>

# BOTH


# Testing on scanned 

In [ ]:
df_copy_editable_1 = df_editable.copy()

In [ ]:
df_copy_editable_1

In [ ]:
df_copy_scanned_1 = df_scanned.copy()

In [ ]:
df_copy_scanned_1

In [ ]:
import re
import unicodedata

In [ ]:
import pandas as pd
import re

def clean_noise(df, filter_keywords=True):
    df_clean = df.copy()

    # === COMPILAR PATRONES DE DIRECCIONES A ELIMINAR ===
    pattern_direccion_completa = re.compile(
        r"(?:Av\.|Jr\.|Calle)\s+[\w\sÁÉÍÓÚáéíóú]+\d{1,5}\s+—\s+Oficina\s+\d{1,4}\s+—\s+[\w\sÁÉÍÓÚáéíóú]+.*",
        flags=re.IGNORECASE
    )
    
    # === PATRÓN DE DIRECCIÓN ESCANEADA O CORRUPTA (completo o incrustado) ===
    pattern_direccion_ruido = re.compile(
        r"""
        (?:
            \b(?:Av|Jr|Calle|Psj|Prolongación)\.?\s+            # Vía
            [\wÁÉÍÓÚáéíóúÑñ\s\.\-]{3,30}                        # Nombre de calle
            \s+\d{3,6}                                          # Número
            (?:\s+[^\s]{3,15})?                                 # Código corrupto: "OfIKi981", etc.
            \s*[-–—]?\s*
            (?:San\s+Isidro|Miraflores|Magdalena|Surco|Jesus\s+María|Lince|Barranco)?
            (?:\s+e\s+)?                                        # Posible "e"
            (?:\.pe|cf\.gob\.pe)?                               # Dominio
        )
        """,
        flags=re.IGNORECASE | re.VERBOSE
    )

    # === CONVERTIR A UNICODE SEGURO ===
    def to_unicode(x):
        try:
            if isinstance(x, bytes):
                return x.decode("utf-8", errors="ignore").strip()
            return str(x).strip()
        except:
            return ""

    df_clean["text"] = df_clean["text"].apply(to_unicode)
    
    # === ELIMINAR FILAS CON DIRECCIONES REPETITIVAS (ANTES DE LIMPIEZAS) ===
    df_clean = df_clean[~df_clean["text"].str.contains(pattern_direccion_completa)].reset_index(drop=True)
    
    # === LIMPIAR O ELIMINAR DIRECCIONES ESCANEADAS ===
    def limpiar_o_eliminar_direccion(text):
        if re.fullmatch(pattern_direccion_ruido, text.strip()):
            return None  # Eliminar párrafo completo
        return re.sub(pattern_direccion_ruido, "", text)  # Reemplazar si está embebido

    df_clean["text"] = df_clean["text"].apply(limpiar_o_eliminar_direccion)
    df_clean = df_clean[df_clean["text"].notnull() & df_clean["text"].str.strip().astype(bool)].reset_index(drop=True)

    # === LIMPIEZAS BÁSICAS VECTORIAL ===
    df_clean["text"] = df_clean["text"].str.replace(r"\b\d{1,3}/\d{1,3}\b", "", regex=True)
    df_clean["text"] = df_clean["text"].str.replace(r"\s+([,;:.!?])", r"\1", regex=True)

    # === LIMPIEZA DE URLS (normales y corruptas) ===
    df_clean["text"] = df_clean["text"].str.replace(
        r"(https?://|htps://|htttp://|htpss://|www\.|wwww\.)[\w\-\.]*\.[a-z]{2,6}(\/[\w\-\.%]*)*", 
        "", regex=True
    )
    df_clean["text"] = df_clean["text"].str.replace(
        r"\b(htps?|htttp|htpss):\/\/[^\s]*", "", regex=True
    )

    df_clean["text"] = df_clean["text"].str.replace(r"[•➢Ø*°¡!?¿\"]", "", regex=True)
    df_clean["text"] = df_clean["text"].str.replace(r":\s*$", ".", regex=True)
    df_clean["text"] = df_clean["text"].str.replace(
        r"Lima[,]?\s+\d{1,2}\s+de\s+[a-záéíóú]+\s+de\s+\d{4}", "", regex=True, flags=re.IGNORECASE)
    df_clean["text"] = df_clean["text"].str.replace("p.p.", "puntos porcentuales")
    df_clean["text"] = df_clean["text"].str.replace(
        r'^\s*((?:[ivxlcdm]+|[a-zA-Z]|\d+)[\.\)]\s*)+', '', regex=True, flags=re.IGNORECASE)
    df_clean["text"] = df_clean["text"].str.replace(
        r"\b(?:a|al|de|del|con|por|para|y|o|en|sin|sobre|ante|tras|entre|hacia|hasta|durante|mediante|excepto|salvo|según)\.$",
        lambda m: m.group(0)[:-1], regex=True)
    
    # === REEMPLAZAR EXPRESIONES CORRUPTAS ===
    pattern_minist = re.compile(r"MINIST\s+ERIO[\w%-º|\"”]+|8N%IC%A\s+Y\s+FIN\s+em.*?Usuaro", flags=re.IGNORECASE)
    pattern_garbage_symbols = re.compile(r"[^\w\sÁÉÍÓÚáéíóúÑñ]{5,}")
    
    # Patrón para capturar bloques muy corruptos (inicio o intermedios)
    pattern_noise_block = re.compile(
        r"(?:[—\[\]_=\]\|%#@*<>~«»]{2,}|[\w]*—[\w]*|—\s*[A-Z]{1}\s*—)+", 
        flags=re.UNICODE
    )
    
    df_clean["text"] = df_clean["text"].str.replace(pattern_minist, "", regex=True)
    df_clean["text"] = df_clean["text"].str.replace(pattern_garbage_symbols, "", regex=True)
    
    # Aplica reemplazo a esos bloques basura
    df_clean["text"] = df_clean["text"].str.replace(pattern_noise_block, "", regex=True)
    
    # Patrón para frases de cortesía y cierre
    pattern_cortesia = re.compile(
        r"(aprovecho\s+la\s+oportunidad\s+para\s+expresar(le)?\s*(a usted)?\s*las?\s*muestras?\s+de\s+mi\s+especial\s+consideración( y estima)?[\.]?)|"  # Variante directa
        r"(le\s+reitero\s+(las?\s+)?muestras?\s+de\s+(mi\s+)?especial\s+consideración[\.]?)|"  # Otra variante frecuente
        r"(hago\s+propicia\s+la\s+ocasión\s+para\s+saludar(lo|la)?\s*(muy)?\s*atentamente[\.]?)|"  # Variante "hago propicia..."
        r"(sin\s+otro\s+particular.*?me\s+despido[^\n\.]*[\.]?)|"  # Frase de cierre clásica
        r"(con\s+(mi\s+)?especial\s+consideración[\.]?)",  # Frase final típica
        flags=re.IGNORECASE
    )

    # Aplicar limpieza al texto
    df_clean["text"] = df_clean["text"].str.replace(pattern_cortesia, "", regex=True)

    # === FILTROS DE TEXTO VÁLIDO ===
    def is_valid(text):
        txt = text.strip()
        letters = [c for c in txt if c.isalpha()]
        if letters and sum(c.isupper() for c in letters) / len(letters) > 0.7:
            return False
        if txt.lower().startswith("fuente:") or re.search(r"\b(Fuente:|Elaboración:)", txt, flags=re.IGNORECASE):
            return False
        if re.search(r"(?:\b\w\s){3,}\w", txt):
            return False
        patterns = [
            r'PCA\s*(Inicial|1er\s*Trim\.|2do\s*Trim\.|3er\s*Trim\.)',
            r'entre\s+\d{4}\sy\s+\d{4}\s*,?\s*\d+\s*de\s+cada\s+100\s*(leyes?|insistencia|implicancia\s*fiscal)',
            r'“[^”]*\(pág\.\s*\d+\)[^”]*”|\(pág\.\s*\d+\)',
            r"Véase informes\s+(N°\s*\d{2}-\d{4}-CF[, y]*){2,}",  # puedes mantenerla si gustas
            r"\b(Véase|Ver|Consultar|Remítase|Revísese)\s+(el\s+)?(informe|artículo|documento|reporte|análisis|dictamen|comunicado\s+oficial|nota)\b.*?(N[°º]?\s*\d{1,3})?.*?\b(del\s+)?\d{4}",
            r"¢\s*US?|US?\s*¢", r"¢"
        ]
        if any(re.search(p, txt, flags=re.IGNORECASE) for p in patterns):
            return False
        if len(txt.split()) < 6:
            return False
        return True

    df_clean = df_clean[df_clean["text"].apply(is_valid)].reset_index(drop=True)

    # === FUSIÓN DE PÁRRAFOS CONSECUTIVOS ===
    rows = []
    i = 0
    while i < len(df_clean):
        current_row = df_clean.iloc[i].copy()
        current_text = current_row["text"]
        current_title = current_row["title"]

        while i + 1 < len(df_clean):
            next_row = df_clean.iloc[i + 1]
            next_text = next_row["text"]
            next_title = next_row["title"]

            if next_title != current_title:
                break

            if not current_text.endswith(".") and next_text and next_text[0].isalpha() and next_text.endswith("."):
                current_text += " " + next_text
                i += 1
                continue

            if not current_text.endswith(".") and re.match(r"^\d{4}", next_text):
                current_text += " " + next_text
                i += 1
                continue

            if re.match(r"^[\(,;:\s]*[a-z\(]", next_text):
                current_text += " " + next_text
                i += 1
                continue

            if not current_text.endswith(".") and re.match(r"^\d{1,3}(?:,\d+)?", next_text):
                current_text += " " + next_text
                i += 1
                continue

            break

        current_row["text"] = current_text
        rows.append(current_row)
        i += 1

    df_result = pd.DataFrame(rows).reset_index(drop=True)
    df_result["paragraph_id"] = df_result.groupby("title").cumcount() + 1

    # === LIMPIEZAS ADICIONALES ===
    cleaned_rows = []
    for idx, row in df_result.iterrows():
        text = row["text"].strip()
        starts_with_special = re.match(r'^[*%\"\'\“\\\[\]\{\}\(\)\^\-+=<>~#@|]', text)
        starts_with_number = re.match(r"^\d+\s+", text)
        prev_text = df_result.iloc[idx - 1]["text"].strip() if idx > 0 else ""
        prev_ends_with_dot = prev_text.endswith(".")

        if starts_with_special:
            continue
        if starts_with_number and prev_ends_with_dot:
            continue

        cleaned_rows.append(row)

    df_result = pd.DataFrame(cleaned_rows).reset_index(drop=True)

    # Eliminar números al inicio del texto aunque estén pegados
    df_result["text"] = df_result["text"].str.replace(r"^\d+\s*", "", regex=True)
    df_result["text"] = df_result["text"].str.replace(r"^\d+(?=\w)", "", regex=True)
    
    # Eliminar observaciones con menos de 75 caracteres
    df_result = df_result[df_result["text"].str.len() >= 75].reset_index(drop=True)

    # === FILTRO OPCIONAL CON KEYWORDS DE ALERTA FISCAL ===
    if filter_keywords:
        keywords = [
            "Incumplimiento de reglas fiscales", "Preocupación", "Advertencia", "Alerta",
            "Riesgos fiscales", "Desvío del déficit fiscal", "No cumplimiento", "Desviaciones significativas",
            "Margen significativo", "Problema de credibilidad fiscal", "Credibilidad fiscal",
            "Sostenibilidad fiscal", "Consolidación fiscal", "Medidas correctivas", "Recomendación",
            "Necesidad de tomar medidas", "Control del gasto público", "Presiones fiscales",
            "Exceso de optimismo en proyecciones", "Exceso de gasto", "Aumento de gasto",
            "Reducción de déficit fiscal", "Incremento de ingresos permanentes",
            "Falta de compromiso con la responsabilidad fiscal", "Medidas de consolidación",
            "Deficiencia en la ejecución del gasto", "Aumento de la deuda pública",
            "Iniciativas legislativas que afectan las finanzas públicas", "Incremento del gasto público",
            "Beneficios tributarios sin justificación", "Tratamientos tributarios preferenciales",
            "Erosión de la base tributaria", "Elusión y evasión tributaria",
            "Aumento de gastos no previstos", "Aumento de gastos extraordinarios",
            "Aumento de gastos en remuneraciones", "Crecimiento del gasto no financiero",
            "Problema de sostenibilidad", "Riesgos de sostenibilidad fiscal", "Aumento de deuda neta",
            "Desajuste fiscal", "Falta de transparencia en el gasto", "Riesgos de sobreendeudamiento",
            "Excepciones a las reglas fiscales", "Riesgo de incumplimiento de metas fiscales",
            "Aumento de los compromisos de deuda", "Riesgo de insolvencia",
            "Falta de flexibilidad fiscal", "Desajuste entre el presupuesto y el MMM",
            "Riesgo de incumplimiento debido a presiones de gasto", "Erosión de la capacidad recaudatoria",
            "Incremento de la deuda pública", "Falta de control de gastos extraordinarios",
            "Necesidad de ajustar el gasto", "Inestabilidad macroeconómica",
            "Problemas fiscales derivados de iniciativas legislativas",
            "Riesgo de desajustes fiscales por reformas",
            "Falta de capacidad de generar ingresos fiscales", "Riesgo de gasto excesivo",
            "Incremento del gasto público no controlado", "Medidas de ajuste fiscal",
            "Inestabilidad presupuestaria", "Riesgo de inestabilidad fiscal",
            "Falta de sostenibilidad de la deuda", "Compromiso con la disciplina fiscal",
            "Necesidad de mejorar la disciplina fiscal", "Riesgos derivados de la crisis financiera",
            "Emergencia fiscal", "No cumplimiento de los límites de deuda",
            "Riesgo de presión sobre las finanzas públicas", "Riesgos de sostenibilidad a largo plazo",
            "Inconsistencia en las proyecciones fiscales", "Proyecciones fiscales no realistas",
            "Implicaciones fiscales de la situación de Petroperú",
            "Desajuste en las proyecciones fiscales", "Necesidad de consolidación fiscal",
            "Riesgos de desequilibrio fiscal", "Amenaza a la estabilidad fiscal",
            "Inseguridad fiscal", "Inconsistencias fiscales", "Falta de previsión en el gasto",
            "Riesgo de pérdida de control fiscal", "Impacto fiscal no anticipado",
            "Presión de gastos adicionales", "Aumento en la presión fiscal",
            "Erosión de las finanzas públicas", "Riesgo de déficit fiscal no controlado",
            "Aumento de la carga fiscal", "Riesgo de crisis fiscal",
            "Propuestas legislativas que generan gasto",
            "Propuestas que limitan la recaudación fiscal",
            "Iniciativas fiscales con implicaciones negativas",
            "Aumento de los gastos sociales no previstos",
            "Riesgo de incumplimiento de los límites fiscales",
            "Propuestas legislativas que no cumplen con las reglas fiscales",
            "Desviaciones fiscales no justificadas",
            "Proyecciones de déficit fiscal no alcanzables",
            "Riesgos derivados de iniciativas legislativas excesivas",
            "Crecimiento de la deuda pública sin control",
            "Necesidad de políticas fiscales más estrictas"
        ]
        keywords_lower = [k.lower() for k in keywords]
        df_result = df_result[
            df_result["text"].str.lower().apply(lambda txt: any(k in txt for k in keywords_lower))
        ].reset_index(drop=True)

    return df_result


In [ ]:
#df_alertas = clean_noise(df, filter_keywords=True)   # Solo alertas fiscales
df_editable_cleaned = clean_noise(df_copy_editable_1, filter_keywords=False) # Todo texto útil

In [ ]:
df_editable_cleaned

In [ ]:
df_copy_ed_2 = df_editable_cleaned[df_editable_cleaned["text"].str.len() <= 75].reset_index(drop=True)
df_copy_ed_2

# Scanned

### The same abve


In [ ]:
df_scanned_cleaned = clean_noise(df_copy_scanned_1, filter_keywords=False)

In [ ]:
df_scanned_cleaned

In [ ]:
df_copy_sc_2 = df_scanned_cleaned[df_scanned_cleaned["text"].str.len() <= 75].reset_index(drop=True)
df_copy_sc_2

# Limpieza global

In [ ]:
raw_text_dataset = pd.read_csv("raw_text_dataset.csv")

In [ ]:
raw_text_dataset

In [ ]:
input_text_dataset = clean_noise(raw_text_dataset, filter_keywords=True)

In [ ]:
input_text_dataset.count()

In [ ]:
# Exportar a CSV con el mismo nombre
input_text_dataset.to_csv("input_text_dataset_true.csv", index=False, encoding="utf-8")

# Limpieza para estadísticos

In [ ]:
# === Carga del DataFrame (si no está cargado) ===
input_text_dataset = pd.read_csv("input_text_dataset_true.csv")

## Order by year and date

In [ ]:
filtered_df = input_text_dataset[input_text_dataset["doc_type"].isna()]
filtered_df

In [ ]:
def sort_by_year_and_date(df):
    """
    Convierte la columna 'date' a datetime (si es necesario) y ordena el DataFrame por 'year' y 'date'.
    
    Parámetros:
        df (pd.DataFrame): DataFrame que contiene las columnas 'year' y 'date'.
    
    Retorna:
        pd.DataFrame: El DataFrame ordenado correctamente por 'year' y 'date'.
    """
    required_cols = {'year', 'date'}
    if not required_cols.issubset(df.columns):
        raise ValueError("El DataFrame debe contener las columnas 'year' y 'date'.")

    # Convertir 'date' a datetime si no lo es ya
    if not pd.api.types.is_datetime64_any_dtype(df["date"]):
        df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors='coerce')

    return df.sort_values(by=["year", "date", "page", "paragraph_id"]).reset_index(drop=True)


In [ ]:
input_text_dataset = sort_by_year_and_date(input_text_dataset)
input_text_dataset

In [ ]:
import pandas as pd

def fill_doc_type_from_title(df):
    """
    Completa los valores vacíos en la columna 'doc_type' a partir del contenido de la columna 'title'.
    Solo extrae 'Comunicado' o 'Informe' si están al inicio del título.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'title' y 'doc_type'.
    
    Retorna:
        pd.DataFrame: El DataFrame con 'doc_type' actualizado donde era nulo.
    """
    # Verificar que las columnas requeridas existan
    required_cols = {'title', 'doc_type'}
    if not required_cols.issubset(df.columns):
        raise ValueError("El DataFrame debe contener las columnas 'title' y 'doc_type'.")

    # Crear máscara de filas donde doc_type está vacío
    mask = df["doc_type"].isna()

    # Expresión regular para capturar solo "Comunicado" o "Informe" al inicio del título
    regex = r"^(Comunicado|Informe)\b"

    # Extraer y llenar solo en filas vacías
    df.loc[mask, "doc_type"] = df.loc[mask, "title"].str.extract(regex)[0]

    return df


In [ ]:
input_text_dataset = fill_doc_type_from_title(input_text_dataset)
input_text_dataset

In [ ]:
filtered_df = input_text_dataset[input_text_dataset["doc_id"].isna()]
filtered_df

In [ ]:
def fill_missing_doc_id(input_text_dataset):
    """
    Llena los valores vacíos en 'doc_id' con números flotantes secuenciales por año, 
    basados en el orden de 'date', empezando en 1.0.

    Parámetros:
        input_text_dataset (pd.DataFrame): DataFrame que debe contener las columnas 'doc_id', 'year', y 'date'.

    Retorna:
        pd.DataFrame: El DataFrame con 'doc_id' actualizado donde era nulo.
    """
    # Verificar que las columnas necesarias existan
    required_cols = {'doc_id', 'year', 'date'}
    if not required_cols.issubset(input_text_dataset.columns):
        raise ValueError("El DataFrame debe contener las columnas 'doc_id', 'year' y 'date'.")

    # Ordenar por year y date
    df = input_text_dataset.sort_values(by=["year", "date"]).copy()

    # Crear máscara de valores vacíos en doc_id
    mask = df["doc_id"].isna()

    # Para cada año, generar un contador incremental para fechas únicas
    def assign_ids(group):
        # Solo para filas con doc_id nulo
        missing = group["doc_id"].isna()
        # Contador secuencial por fecha
        group.loc[missing, "doc_id"] = (
            group.loc[missing]
                 .groupby("date", sort=False)
                 .ngroup() + 1
        ).astype(float)
        return group

    df = df.groupby("year", group_keys=False).apply(assign_ids)

    return df


In [ ]:
input_text_dataset = fill_missing_doc_id(input_text_dataset)
input_text_dataset

# TO csv

In [ ]:
# Exportar a CSV con el mismo nombre
input_text_dataset.to_csv("llm_input_text_dataset_true.csv", index=False, encoding="utf-8")

## Estadísticos de párrafos

In [ ]:
import pandas as pd

# === Carga del DataFrame (si no está cargado) ===
# input_text_dataset = pd.read_csv("input_text_dataset.csv")

# === Total de documentos únicos, por tipo ===
total_docs = input_text_dataset[['title', 'doc_id', 'date']].apply(tuple, axis=1).nunique()
docs_por_tipo = input_text_dataset.drop_duplicates('title')['doc_type'].value_counts()

In [ ]:
total_docs

In [ ]:
docs_por_tipo

In [ ]:
# === Años con más y menos opiniones ===
opiniones_por_año = input_text_dataset.groupby('year')['doc_id'].nunique()
año_mas_opiniones = opiniones_por_año.idxmax()
año_menos_opiniones = opiniones_por_año.idxmin()

In [ ]:
opiniones_por_año

In [ ]:
año_mas_opiniones

In [ ]:
año_menos_opiniones

In [ ]:
# === Documento con más/menos párrafos ===
parrafos_por_doc = input_text_dataset.groupby('title')['paragraph_id'].nunique()
doc_mas_parrafos = parrafos_por_doc.idxmax()
doc_menos_parrafos = parrafos_por_doc.idxmin()

In [ ]:
doc_mas_parrafos

In [ ]:
doc_menos_parrafos

In [ ]:
parrafo_mas_pequeno = input_text_dataset.loc[input_text_dataset["text_length"].idxmin()]
parrafo_mas_grande = input_text_dataset.loc[input_text_dataset["text_length"].idxmax()]
tamano_promedio_parrafo = input_text_dataset["text_length"].mean()

In [ ]:
parrafo_mas_pequeno

In [ ]:
parrafo_mas_grande

In [ ]:
tamano_promedio_parrafo

# palabras


In [ ]:
# === Conteo de palabras por párrafo ===
input_text_dataset["word_count"] = input_text_dataset["text"].str.split().str.len()

# Párrafo con menor cantidad de palabras
parrafo_mas_pequeno = input_text_dataset.loc[input_text_dataset["word_count"].idxmin()]

# Párrafo con mayor cantidad de palabras
parrafo_mas_grande = input_text_dataset.loc[input_text_dataset["word_count"].idxmax()]

# Tamaño promedio de párrafo en número de palabras
tamano_promedio_parrafo = input_text_dataset["word_count"].mean()


In [ ]:
parrafo_mas_pequeno

In [ ]:
parrafo_mas_grande

In [ ]:
tamano_promedio_parrafo

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: #033280; font-weight: bold;">
        <a href="#outilne" style="color: #033280; text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: #cd301b; text-decoration: none;">Back to the outline.</a>
</div>